# UserForGenre

Definir una función que identifique el usuario con la mayor cantidad de horas jugadas en el género especificado y proporcione una lista que muestre la acumulación de horas jugadas por año.

def UserForGenre( genero : str ): 
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

### Importamos las librerías

In [1]:
import pandas as pd
import ast

### Lectura de los archivos CSV

In [2]:
steam_games = pd.read_csv("..\\Datsets\\CSV\\steam_games.csv")
user_items =pd.read_csv("..\\Datsets\\CSV\\user_items.csv")

La información necesaria para esta consulta se encuentra en los dataframes `steam_games` y `user_items`. Del dataframe `steam_games`, requerimos las columnas 'genres' y 'release_year', mientras que del dataframe `user_items` necesitamos los datos de 'playtime_forever' y 'user_id'. Para lograr esto, procederemos a concatenar los dataframes utilizando la columna 'item_id'.

In [3]:
# Unir los DataFrames usando las columnas 'id' en steam_games y 'item_id' en user_items
df_UserForGenre = pd.merge(user_items, steam_games, left_on='item_id', right_on='id', how='inner')

# Seleccionar las columnas requeridas
df_UserForGenre = df_UserForGenre[['genres', 'release_year', 'playtime_forever', 'user_id']]

df_UserForGenre.head()

,genres,release_year,playtime_forever,user_id
0,"['Action', 'Strategy']",2000.0,6,76561197970982479
1,"['Action', 'Strategy']",2000.0,0,js41637
2,"['Action', 'Strategy']",2000.0,0,Riot-Punch
3,"['Action', 'Strategy']",2000.0,93,doctr
4,"['Action', 'Strategy']",2000.0,108,corrupted_soul


Expandimos las listas de la columna genres en filas.

In [4]:
# Verificar si 'genres' contiene NaN antes de aplicar ast.literal_eval
df_UserForGenre['genres'] = df_UserForGenre['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
#Expandimos las listas en columnas
df_UserForGenre = df_UserForGenre.explode('genres').reset_index(drop=True)
df_UserForGenre.head()

,genres,release_year,playtime_forever,user_id
0,Action,2000.0,6,76561197970982479
1,Strategy,2000.0,6,76561197970982479
2,Action,2000.0,0,js41637
3,Strategy,2000.0,0,js41637
4,Action,2000.0,0,Riot-Punch


Buscamos al usuario que tiene la mayor cantidad de horas jugadas para cada género

In [5]:
# Encontrar el user_id con la mayor cantidad de horas jugadas por género
jugador_mas_horas = df_UserForGenre.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()

# Encontrar el índice del user_id con la mayor cantidad de horas jugadas por género
indice = jugador_mas_horas.groupby('genres')['playtime_forever'].idxmax()

# Filtrar el DataFrame original usando los índices encontrados
jugador_mas_horas = jugador_mas_horas.loc[indice]

In [6]:
jugador_mas_horas.head()

,genres,user_id,playtime_forever
51298,Action,Sp3ctre,1978403
117882,Adventure,REBAS_AS_F-T,2700123
136676,Audio Production,Lickidactyl,109916
181416,Casual,REBAS_AS_F-T,1521355
199976,Early Access,76561197978756659,316969


Concatenamos los DataFrames jugador_mas_horas y df_UserForGenre para retener únicamente al jugador que ha acumulado la mayor cantidad de horas por género del DataFrame df_UserForGenre. Además, conservamos la información sobre la cantidad de horas jugadas por año de lanzamiento.

In [7]:
# Concatenamos los DataFrames
df_UserForGenre = pd.merge(df_UserForGenre, jugador_mas_horas[['genres', 'user_id']], on=['genres','user_id'])
df_UserForGenre.head()

,genres,release_year,playtime_forever,user_id
0,Action,2000.0,70644,Sp3ctre
1,Action,1999.0,44,Sp3ctre
2,Action,1999.0,0,Sp3ctre
3,Action,1998.0,0,Sp3ctre
4,Action,2001.0,13,Sp3ctre


Filtramos los valores para eliminar las entradas donde la cantidad de horas jugadas (playtime_forever) es igual a 0.

In [8]:
df_UserForGenre = df_UserForGenre[df_UserForGenre['playtime_forever']>0]
df_UserForGenre.head()

,genres,release_year,playtime_forever,user_id
0,Action,2000.0,70644,Sp3ctre
1,Action,1999.0,44,Sp3ctre
4,Action,2001.0,13,Sp3ctre
5,Action,2010.0,249,Sp3ctre
6,Action,2004.0,122426,Sp3ctre


Agrupamos por release_year

In [9]:
df_UserForGenre = df_UserForGenre.groupby(['genres', 'release_year','user_id'])['playtime_forever'].sum().reset_index()
df_UserForGenre.head()

,genres,release_year,user_id,playtime_forever
0,Action,1995.0,Sp3ctre,217
1,Action,1997.0,Sp3ctre,242
2,Action,1999.0,Sp3ctre,44
3,Action,2000.0,Sp3ctre,70644
4,Action,2001.0,Sp3ctre,250


### Convertimos a CSV

In [10]:
#Convertimos a CSV
df_UserForGenre.to_csv('..\\Datsets\\Archivos_API\\UserForGenre.csv', index=False)

### Función

In [11]:
def UserForGenre(genre):
    # Filtrar el DataFrame por el género dado
    genero = df_UserForGenre[df_UserForGenre['genres'] == genre]

    # Crear una lista de diccionarios para las horas jugadas por año
    horas_anio = [{'Año': int(year), 'Horas': int(playtime)} for year, playtime in genero[['release_year', 'playtime_forever']].values]

    # Crear el diccionario de salida
    resultado = {
        'Usuario con más horas jugadas para género '+ genre: genero.iloc[0]['user_id'],
        "Horas jugadas": horas_anio
    }

    return resultado


In [12]:
genre_result = UserForGenre('Action')
print(genre_result)

{'Usuario con más horas jugadas para género Action': 'Sp3ctre', 'Horas jugadas': [{'Año': 1995, 'Horas': 217}, {'Año': 1997, 'Horas': 242}, {'Año': 1999, 'Horas': 44}, {'Año': 2000, 'Horas': 70644}, {'Año': 2001, 'Horas': 250}, {'Año': 2002, 'Horas': 238}, {'Año': 2003, 'Horas': 7744}, {'Año': 2004, 'Horas': 127411}, {'Año': 2005, 'Horas': 21339}, {'Año': 2006, 'Horas': 90253}, {'Año': 2007, 'Horas': 112794}, {'Año': 2008, 'Horas': 8533}, {'Año': 2009, 'Horas': 108771}, {'Año': 2010, 'Horas': 82766}, {'Año': 2011, 'Horas': 155459}, {'Año': 2012, 'Horas': 384238}, {'Año': 2013, 'Horas': 122158}, {'Año': 2014, 'Horas': 132864}, {'Año': 2015, 'Horas': 389314}, {'Año': 2016, 'Horas': 30195}, {'Año': 2017, 'Horas': 43327}]}
